In [ ]:
import os
import re
import pandas as pd
import numpy as np

import tensorflow
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model , Sequential
from tensorflow.keras.layers import LSTM , Embedding , TimeDistributed , Dropout , GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as k
from keras.layers.recurrent import LSTM 
from keras.layers.core import activations
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix , accuracy_score

Using TensorFlow backend.


In [ ]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install flair

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
     |████████████████████████████████| 704.8MB 21kB/s 
     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 983kB 8.8MB/s 
     |████████████████████████████████| 778kB 15.2MB/s 
     |████████████████████████████████| 798kB 22.7MB/s 
     |████████████████████████████████| 276kB 30.0MB/s 
     |████████████████████████████████| 1.1MB 27.7MB/s 
     |████████████████████████████████| 3.0MB 36.9MB/s 
     |████████████████████████████████| 890kB 42.9MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.6.0-cp36-none-any.whl size=14689 sha256=2bb1cce4c74a346e173d6f8679650073e0126138c20eb908ef9702aa5a881df5
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=e1a655e5365e5346438c9023c686511d06f6ace546add809519beb

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

**Read DataSet**

In [ ]:
x_train = open('train.txt','r').readlines()
lable_train = x_train[1]
print(len(x_train))

251574


In [ ]:
from collections import Counter
count = Counter(x_train)
count = dict(count)
print(count)

{'مدیران|O\n': 52, 'چهارمین|O\n': 15, 'جشنواره|O\n': 123, 'فیلم|O\n': 119, 'و|O\n': 9641, 'عکس|O\n': 36, 'فناوری|O\n': 57, 'صنعتی|O\n': 53, 'معرفی|O\n': 78, 'شدند|O\n': 122, '\n': 8028, 'به|O\n': 6374, 'گزارش|O\n': 621, 'خبرگزاری|B_ORG\n': 192, 'مهر|I_ORG\n': 46, '،|O\n': 6311, 'سعید|B_PER\n': 27, 'پورعلی|I_PER\n': 1, 'معاون|O\n': 148, 'فرهنگی|O\n': 176, 'جهاد|B_ORG\n': 13, 'دانشگاهی|I_ORG\n': 6, 'کشور|I_ORG\n': 43, 'با|O\n': 3645, 'تاکید|O\n': 276, 'بر|O\n': 805, 'اینکه|O\n': 540, 'سینما|O\n': 33, 'یکی|O\n': 276, 'از|O\n': 4925, 'موثرترین|O\n': 2, 'روش های|O\n': 6, 'فرهنگ سازی|O\n': 12, 'در|O\n': 8731, 'حوزه|O\n': 197, 'صنعت|O\n': 58, 'تولید|O\n': 232, 'است|O\n': 3140, 'گفت|O\n': 1002, ':|O\n': 3022, 'توجه|O\n': 243, 'گستردگی|O\n': 4, 'روزافزون|O\n': 1, 'کشور|O\n': 702, 'این|O\n': 4148, 'دو|O\n': 430, 'نمی توانند|O\n': 12, 'یکدیگر|O\n': 33, 'کناره گیری|O\n': 10, 'کنند|O\n': 275, '.|O\n': 6676, 'چه بسا|O\n': 3, 'امروز|O\n': 108, 'کشورهای|O\n': 134, 'پیشرفته|O\n': 15, 'جایگاه|O\n': 45, 

In [ ]:
# Convert dataframes to numpy arrays
x_train = np.asarray(x_train)
lable_train = np.asarray(lable_train)

**Preprocessing**

In [ ]:
pip install normalization

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for normalization: filename=normalization-0.4-cp36-none-any.whl size=44016 sha256=e08528ead90505b5fcbcf9d914d07a8cff97096e1b7b927854662aa4e25864c7
  Stored in directory: /root/.cache/pip/wheels/41/82/64/5efe898b7aaf4ba436f813c747e1c7586fc49341e2d689c8de
Successfully built normalization


In [ ]:
from normalization import normalize
normalize = normalize('')

In [ ]:
for token in x_train:
    print(token)

Streaming output truncated to the last 5000 lines.

ایشان|O

داده|O

شده|O

،|O

گفت|O

:|O

در|O

قزاقستان|B_LOC

،|O

هدف|O

دشمنان|O

،|O

دور|O

کردن|O

مردم|O

از|O

فرهنگ|O

و|O

گذشته ی|O

آنهاست|O

.|O



از|O

این|O

رو|O

شاهد|O

گسترش|O

زبان|O

-|O

های|O

بیگانه|O

در|O

آن|O

کشور|O

هستیم|O

و|O

بنا|O

به|O

همین|O

دلیل|O

،|O

گرایش|O

به|O

زبان|O

فارسی|O

کمتر|O

شده|O

است|O

،|O

حال آنکه|O

تاریخ|O

و|O

فرهنگ|O

ملت|O

قزاقستان|B_LOC

با|O

فرهنگ|O

و|O

تمدن|O

ایرانی|O

گره|O

خورده|O

است|O

و|O

ما|O

معتقدیم|O

زبان|O

فارسی|O

،|O

برای|O

ما|O

یک|O

زبان|O

بیگانه|O

نیست|O

و|O

از|O

ماست|O

.|O



وی|O

با|O

اشاره|O

به|O

دامنه|O

گسترش|O

زبان|O

فارسی|O

در|O

بین|O

متخصصان|O

و|O

عامه|O

مردم|O

در|O

گذشته|O

،|O

خاطر نشان|O

کرد|O

:|O

زبان|O

فارسی|O

زبان|O

علم|O

،|O

ادب|O

و|O

فرهنگ|O

بوده|O

است|O

و|O

آثار|O

بزرگانی|O

مانند|O

سعدی|B_PER

و|O

فردوسی|B_PER

هنوز|O

در|O

بین|O

مردم|O

جایگاه|O

خود|O

را|O

حفظ|O

کرده|O

است

In [ ]:
tokens=[]
for token in x_train:
  token = re.sub("()/+]", " ", token) 
  token = re.sub(r"\!+","!", token)
  token = re.sub(r"\"","؟", token)
  tokens.append(token)

#print(tokens)

In [ ]:
from flair.models import SequenceTagger
tagger = SequenceTagger.load('ner')

2020-07-30 08:21:04,711 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/models-v0.4/NER-conll03-english/en-ner-conll03-v0.4.pt not found in cache, downloading to /tmp/tmp74t1nl2j


100%|██████████| 432197603/432197603 [00:17<00:00, 25022520.53B/s]

2020-07-30 08:21:22,508 copying /tmp/tmp74t1nl2j to cache at /root/.flair/models/en-ner-conll03-v0.4.pt


2020-07-30 08:21:23,774 removing temp file /tmp/tmp74t1nl2j
2020-07-30 08:21:23,838 loading file /root/.flair/models/en-ner-conll03-v0.4.pt


In [ ]:
sentence = Sentence('')

# predict NER tags
tagger.predict(sentence)

# print sentence with predicted tags
print(sentence.to_tagged_string())

2020-07-30 08:21:35,603 Warning: An empty Sentence was created! Are there empty strings in your dataset?

